In [1]:
from copy import deepcopy
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

import matplotlib.pyplot as plt

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

Version:  2.9.2
Eager mode:  True
Hub version:  0.12.0
GPU is available


In [2]:
train_data, test_data = tfds.load(name="imdb_reviews", split=["train", "test"], 
                                    batch_size=-1, as_supervised=True)

Metal device set to: Apple M1 Pro

systemMemory: 32.00 GB
maxCacheSize: 10.67 GB



2022-10-14 09:15:07.607455: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


In [3]:
from flex.data import FlexDataObject

flex_data = FlexDataObject.from_tfds_dataset(train_data)

In [4]:
from flex.data import FlexDatasetConfig, FlexDataDistribution

config = FlexDatasetConfig(seed=0)
config.n_clients = 2
config.replacement = False # ensure that clients do not share any data
config.client_names = ['client1', 'client2'] # Optional
flex_dataset = FlexDataDistribution.from_config(cdata=flex_data, config=config)

In [5]:
from flex.data import FlexDataDistribution

flex_dataset = FlexDataDistribution.iid_distribution(flex_data, n_clients=2)

In [6]:
from flex.pool.flex_primitives import deploy_model_to_clients
from flex.pool.flex_primitives import evaluate_model

from flex.pool.flex_decorators import init_server_model_decorator
from flex.pool.flex_decorators import collector_decorator
from flex.pool.flex_decorators import aggregator_decorator_tf
from flex.pool.flex_decorators import train_decorator
from flex.pool.flex_decorators import deploy_decorator

In [7]:
# Defining the model
@init_server_model_decorator
def define_model(*args):
    model = "https://tfhub.dev/google/nnlm-en-dim50-with-normalization/2"
    hub_layer = hub.KerasLayer(model, input_shape=[], dtype=tf.string, trainable=True)
    model = tf.keras.Sequential()
    model.add(hub_layer)
    model.add(tf.keras.layers.Dense(16, activation='relu'))
    model.add(tf.keras.layers.Dense(1))
    model.compile(optimizer='adam',
                    loss=tf.losses.BinaryCrossentropy(from_logits=True),
                    metrics=[tf.metrics.BinaryAccuracy(threshold=0.0, name='accuracy')])
    return model

In [8]:
from flex.pool import FlexPool

flex_pool = FlexPool.client_server_architecture(fed_dataset=flex_dataset, init_func=define_model, verbose=1, model_params=[])

Initializing server model.


In [9]:
clients = flex_pool.clients
server = flex_pool.servers
print(f"Server node is indentified by {server.actor_ids}")
print(f"Client nodes are identified by {clients.actor_ids}")

Server node is indentified by ['server']
Client nodes are identified by ['client_0', 'client_1']


In [10]:
server.map(deploy_model_to_clients, clients, verbose=1)

Initializing model at a client
INFO:tensorflow:Assets written to: ram://2c966620-921b-4da1-9dee-9298821abbfd/assets


INFO:tensorflow:Assets written to: ram://2c966620-921b-4da1-9dee-9298821abbfd/assets


INFO:tensorflow:Assets written to: ram://cde47a77-d89f-4a4e-8ab4-a176f4d9496e/assets


INFO:tensorflow:Assets written to: ram://cde47a77-d89f-4a4e-8ab4-a176f4d9496e/assets


[None]

In [11]:
@train_decorator
def fit_tf(model, X_data, y_data, *args, **kwargs):
    model.fit(X_data, y_data, *args, **kwargs)

In [12]:
clients.map(fit_tf, batch_size=512, epochs=2)

Epoch 1/2
25/25 [==============================] - 2s 77ms/step - loss: 0.6487 - accuracy: 0.6361
Epoch 2/2
25/25 [==============================] - 2s 74ms/step - loss: 0.5254 - accuracy: 0.7847
Epoch 1/2
25/25 [==============================] - 2s 76ms/step - loss: 0.6485 - accuracy: 0.6409
Epoch 2/2
25/25 [==============================] - 2s 74ms/step - loss: 0.5259 - accuracy: 0.7845


[None, None]

In [13]:
aggregator = flex_pool.aggregators
aggregator.actor_ids

['server']

In [14]:
@collector_decorator
def tensorflow_weights_collector(client_model):
    return client_model.get_weights()

In [15]:
clients.map(tensorflow_weights_collector, aggregator)

[None, None]

In [16]:
@aggregator_decorator_tf
def fed_avg(agg_model):
    return np.mean(np.array(agg_model), axis=0)

In [17]:
aggregator.map(fed_avg, verbose=1)

Aggregating weights


/var/folders/9d/8sk2j5nd30v6mvx2vx61hp980000gn/T/ipykernel_89200/3754503321.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.mean(np.array(agg_model), axis=0)


[None]

In [18]:
@deploy_decorator
def deploy_global_model(server_model, clients_models):
    aggregated_weights = server_model.get_weights()
    for client_model in clients_models:
        clients_models[client_model]["model"].set_weights(aggregated_weights)

In [19]:
server.map(deploy_global_model, clients)

Args: [{'model': <keras.engine.sequential.Sequential object at 0x17b581550>, 'weights': []}, {'client_0': {'model': <keras.engine.sequential.Sequential object at 0x299bcb640>}, 'client_1': {'model': <keras.engine.sequential.Sequential object at 0x298de5310>}}]


[None]

In [20]:
test_examples, test_labels = test_data

In [21]:
server.map(evaluate_model, test_examples=test_examples, test_labels=test_labels)

Evaluating model at server
782/782 [==============================] - 6s 8ms/step - loss: 0.4984 - accuracy: 0.7852
Results on test data: [0.4984446167945862, 0.7852000594139099]


[None]

In [22]:
clients.map(evaluate_model, test_examples=test_examples, test_labels=test_labels)

Evaluating model at client.
391/391 [==============================] - 3s 8ms/step - loss: 0.4595 - accuracy: 0.8281
Results at client on client's data: [0.4594801962375641, 0.8280800580978394]
782/782 [==============================] - 6s 8ms/step - loss: 0.4984 - accuracy: 0.7852
Results on test data: [0.4984446167945862, 0.7852000594139099]
Evaluating model at client.
391/391 [==============================] - 3s 8ms/step - loss: 0.4588 - accuracy: 0.8246
Results at client on client's data: [0.45879843831062317, 0.8245600461959839]
782/782 [==============================] - 6s 8ms/step - loss: 0.4984 - accuracy: 0.7852
Results on test data: [0.4984446167945862, 0.7852000594139099]


[None, None]

# Putting it all together

In [23]:
def train_n_rounds(n_rounds, batch_size, epochs):
    pool = FlexPool.client_server_architecture(fed_dataset=flex_dataset, init_func=define_model, verbose=1, model_params=[])
    pool.servers.map(deploy_model_to_clients, pool.clients, verbose=1)
    for i in range(n_rounds):
        print(f"\nRunning round: {i}\n")
        pool.clients.map(fit_tf, batch_size=batch_size, epochs=epochs, verbose=1)
        pool.clients.map(evaluate_model, test_examples=test_examples, test_labels=test_labels)
        pool.clients.map(tensorflow_weights_collector, pool.aggregators, verbose=1)
        pool.aggregators.map(fed_avg, verbose=1)
        pool.servers.map(deploy_global_model, pool.clients)
        pool.servers.map(evaluate_model, test_examples=test_examples, test_labels=test_labels)

In [24]:
train_n_rounds(n_rounds=4, batch_size=512, epochs=10)

Initializing server model.
Initializing model at a client
INFO:tensorflow:Assets written to: ram://88c1fecd-ab25-475e-a4e6-2d8f330efcb8/assets


INFO:tensorflow:Assets written to: ram://88c1fecd-ab25-475e-a4e6-2d8f330efcb8/assets


INFO:tensorflow:Assets written to: ram://5d668fa2-bd23-474c-8e51-c6962a118fa6/assets


INFO:tensorflow:Assets written to: ram://5d668fa2-bd23-474c-8e51-c6962a118fa6/assets



Running round: 0

Training model at client.
Epoch 1/10
25/25 [==============================] - 3s 91ms/step - loss: 0.6946 - accuracy: 0.5522
Epoch 2/10
25/25 [==============================] - 2s 86ms/step - loss: 0.5706 - accuracy: 0.7770
Epoch 3/10
25/25 [==============================] - 2s 83ms/step - loss: 0.4400 - accuracy: 0.8510
Epoch 4/10
25/25 [==============================] - 2s 81ms/step - loss: 0.3188 - accuracy: 0.9011
Epoch 5/10
25/25 [==============================] - 2s 81ms/step - loss: 0.2305 - accuracy: 0.9287
Epoch 6/10
25/25 [==============================] - 2s 82ms/step - loss: 0.1708 - accuracy: 0.9497
Epoch 7/10
25/25 [==============================] - 2s 81ms/step - loss: 0.1266 - accuracy: 0.9687
Epoch 8/10
25/25 [==============================] - 2s 79ms/step - loss: 0.0941 - accuracy: 0.9808
Epoch 9/10
25/25 [==============================] - 2s 83ms/step - loss: 0.0697 - accuracy: 0.9882
Epoch 10/10
25/25 [==============================] - 2s 79ms/ste

/var/folders/9d/8sk2j5nd30v6mvx2vx61hp980000gn/T/ipykernel_89200/3754503321.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.mean(np.array(agg_model), axis=0)


Evaluating model at server
782/782 [==============================] - 7s 9ms/step - loss: 0.2963 - accuracy: 0.8799
Results on test data: [0.296309232711792, 0.8799200654029846]

Running round: 1

Training model at client.
Epoch 1/10
25/25 [==============================] - 3s 96ms/step - loss: 0.0707 - accuracy: 0.9804
Epoch 2/10
25/25 [==============================] - 2s 91ms/step - loss: 0.0467 - accuracy: 0.9929
Epoch 3/10
25/25 [==============================] - 2s 100ms/step - loss: 0.0327 - accuracy: 0.9968
Epoch 4/10
25/25 [==============================] - 2s 86ms/step - loss: 0.0239 - accuracy: 0.9983
Epoch 5/10
25/25 [==============================] - 2s 87ms/step - loss: 0.0181 - accuracy: 0.9990
Epoch 6/10
25/25 [==============================] - 2s 83ms/step - loss: 0.0140 - accuracy: 0.9995
Epoch 7/10
25/25 [==============================] - 2s 82ms/step - loss: 0.0112 - accuracy: 0.9997
Epoch 8/10
25/25 [==============================] - 2s 87ms/step - loss: 0.0091 - a

# END